# ID3 Algorithm for Machine Learning 
## Using Real League of Legends Statistics from 2023




### Data Reading and Filtering:
The code reads data from an original CSV file containing LoL esports match data. This dataset offers a rich source of information for analyzing team strategies, player performances, and overall game dynamics. The subsequent code refines and structures this data, emphasizing champion-related details, and ultimately saves the processed information for further exploration and analysis in a new CSV file named 'champions_info.csv'.
It filters the data to include only matches from specified leagues (LEC, LCK, LPL, LCS).

Removes rows with empty 'champion' or 'team' in the 'side' column.

In [1]:
### For reference
import pandas as pd
import random
import numpy as np
from sklearn.tree import DecisionTreeClassifier, export_text

#data = pd.read_csv('2023_LoL_esports_match_data_from_OraclesElixir.csv', low_memory=False)
url = 'https://raw.githubusercontent.com/JoxtaQrst/Machine-Learning/main/ID3-Fabian/2023_LoL_esports_match_data_from_OraclesElixir.csv'

data = pd.read_csv(
    url,
    sep=',',
    encoding='utf-8',
    low_memory=False
)

# print(data.columns.values)
print(data.iloc[0])

gameid              ESPORTSTMNT06_2753012
datacompleteness                 complete
url                                   NaN
league                               LFL2
year                                 2023
                            ...          
assistsat15                           0.0
deathsat15                            0.0
opp_killsat15                         0.0
opp_assistsat15                       0.0
opp_deathsat15                        0.0
Name: 0, Length: 123, dtype: object


### Column Classification:
Selects relevant columns ('champion', 'league', 'result', 'kills', 'deaths', 'assists', 'side', 'damagetochampions', 'position').

Reads additional champion attributes from a CSV file ('fabian.csv') and merges it with the selected columns based on the 'champion' column.

Creates a binary column for each role ('top', 'mid', 'jng', 'sup', 'bot') based on the 'position' column.
Removes rows with missing values in the 'position' column.


### Data Shuffling and Limiting:
Shuffles the data randomly within each league group.

Limits each league to a maximum of 75 rows.


### Derived Column Addition and Data Saving:
Drops the 'position' column from the data.

Creates a new column 'first_blood_kill' based on conditions involving 'firstblood' and 'kills'.

Saves the processed data to a new CSV file named 'champions_info.csv'.

Prints a subset of the original data containing 'firstblood' and 'firstbloodkill' columns.

## 1. Preprocessing
### Provide a brief description of the dataset
The dataset contains information related to champions League of Legends. Attributes include champion names, league, results, side, difficulty, items for spike, attack type, roles (top, mid, jng, sup, bot), and various numerical statistics like kills, deaths, assists, and damage to champions.

In [2]:
import pandas as pd
import math
from pprint import pprint

from sklearn.svm._libsvm import predict
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

#data = pd.read_csv('champions_info.csv')
url = 'https://raw.githubusercontent.com/JoxtaQrst/Machine-Learning/main/ID3-Fabian/champions_info.csv'

data = pd.read_csv(
    url,
    sep=',',
    encoding='utf-8',
    low_memory=False
)
# b. Drop unnecessary columns
data = data.dropna()

### a. What are the attributes, what is the target attribute?

In [3]:
attributes = data.columns[:-1]  # all columns except the last one
target_attribute = 'first_blood_kill'

print(f"Attributes: {attributes}")
print(f"\nTarget Attribute: {target_attribute}")

Attributes: Index(['champion', 'league', 'result', 'kills', 'deaths', 'assists', 'side',
       'damagetochampions', 'Difficulty', 'Items_For_Spike', 'Attack_Type',
       'top', 'mid', 'jng', 'sup', 'bot'],
      dtype='object')

Target Attribute: first_blood_kill


In our case , we will ignore the 'champion' collumn as it only gives us the name of the champion.

### What is the purpose of the dataset?
To predict whether the champion did the first blood or not.
### Specify which attributes are discrete and continuous.



In [4]:
discrete_attributes = ['league', 'result', 'side', 'Difficulty', 'Items_For_Spike', 'Attack_Type', 'top',
                       'mid', 'jng', 'sup', 'bot']
continuous_attributes = ['kills', 'deaths', 'assists', 'damagetochampions']

print(f"Discrete Attributes: {discrete_attributes}")
print(f"\nContinuous Attributes: {continuous_attributes}")

Discrete Attributes: ['league', 'result', 'side', 'Difficulty', 'Items_For_Spike', 'Attack_Type', 'top', 'mid', 'jng', 'sup', 'bot']

Continuous Attributes: ['kills', 'deaths', 'assists', 'damagetochampions']


### b. Identify the NaN's (Not a Number) in your dataset. Remove the rows that contain such values.
In our dataset, there are no NaN numbers.

### c. Calculate the mean and variance for each numerical attribute
In our case, the numerical attributes are the continous_attributes

In [5]:
mean_variances = {}
for attribute in continuous_attributes:
    mean_variances[attribute] = [data[attribute].mean(), data[attribute].var()]

print(mean_variances)

{'kills': [2.796666666666667, 6.79129319955407], 'deaths': [2.42, 3.241070234113713], 'assists': [6.173333333333333, 21.28089186176143], 'damagetochampions': [14555.59, 82937678.89153846]}


## 2. Probabilities an Information Theory
### a. Write a function compute_probabilities that calculates the probability mass function of a discrete attribute.
For example let's calculate the probabilities of 'league' values

In [6]:
def compute_probabilities(data, attribute):
    return data[attribute].value_counts(normalize=True).to_dict()


# e.g. compute_probabilities(data, 'league')
probabilites = compute_probabilities(data, 'league')
print("Probabilities : ", probabilites)

Probabilities :  {'LCK': 0.25, 'LCS': 0.25, 'LEC': 0.25, 'LPL': 0.25}


### b. Write a function calculate_entropy that computes the entropy of a random variable given its probability distribution. Calculate the entropy for each discrete attribute.
For example , let's calculate the entropy for 'league'

In [7]:
def calculate_entropy(data, attribute):
    probabilities = compute_probabilities(data, attribute)
    entropy = -sum(p * math.log2(p) for p in probabilities.values())
    return entropy


# e.g. calculate_entropy(data, 'league')
entropy_league = calculate_entropy(data, 'league')
print("Entropy : ", entropy_league)

Entropy :  2.0


### c. Write a function calculate_conditional_entropy that computes the conditional entropy of two random variables. Calculate H(Y |X), where Y denotes the target attribute, and X one of the discrete attributes from the dataset.
For example, let's calculate H('first_blood_kill'|'league')

In [8]:
def calculate_conditional_entropy(data, attribute, target_attribute):
    conditional_entropy = 0
    for attribute_values in data[attribute].unique():
        subset = data[data[attribute] == attribute_values]
        probability_attribute_values = len(subset) / len(data)
        entropy_attribute_value = calculate_entropy(subset, target_attribute)
        conditional_entropy += probability_attribute_values * entropy_attribute_value
    return conditional_entropy


# e.g. calculate_conditional_entropy(data, 'league', 'first_blood_kill')
conditional_entropy = calculate_conditional_entropy(data, 'league', 'first_blood_kill')
print("Condititonal Entropy : ", conditional_entropy)

Condititonal Entropy :  0.6205651394665301


### d. Write a function calculate_information_gain that computes the information gain of two random variables. Calculate the information gain in the previous example.
IG('first_blood_kill'|'league')

In [9]:
def calculate_information_gain(data, attribute, target_attribute):
    target_entropy = calculate_entropy(data, target_attribute)
    conditional_entropy = calculate_conditional_entropy(data, attribute, target_attribute)
    information_gain = target_entropy - conditional_entropy
    return information_gain


# e.g. calculate_information_gain(data, 'league', 'first_blood_kill')
information_gain = calculate_information_gain(data, 'league', 'first_blood_kill')
print("Information Gain : ", information_gain)

Information Gain :  0.005710049607652756


# 3. ID3
## a. Write a function find_root_node that finds the attribute picked by ID3 as root. The function should return a tuple with the name of the attribute and 
the information gain. Use the functions created at point 2.? 

In [10]:
def find_root_node(data, attributes, target_attribute):
    information_gains = [(attribute, calculate_information_gain(data, attribute, target_attribute)) for attribute in
                         attributes]
    best_attribute, best_information_gain = max(information_gains, key=lambda x: x[1])
    return best_attribute, best_information_gain

### What is the attribute identified as a root node?

In [11]:
root_node, root_ig = find_root_node(data, discrete_attributes, 'first_blood_kill')
print("Root Node:", root_node, "\nInformation Gain:", root_ig)

Root Node: Items_For_Spike 
Information Gain: 0.010141885489185931


## b. Write a function id3_discrete that implements the ID3 algorithm for the discrete attributes

In [12]:
def id3_discrete(data, discrete_attributes, target_attribute):
    # if all target attributes have the same value, return that value
    if len(data[target_attribute].unique()) == 1:
        return data[target_attribute].unique()[0]

    # if data is empty, return the most common value of the target attribute
    if len(data) == 0:
        return data[target_attribute].value_counts().idxmax()

    # if there are no attributes left, return the most common target attribute value
    if len(discrete_attributes) <= 1:
        return data[target_attribute].value_counts().idxmax()

    # Choose the attribute with the highest information gain
    root_attribute, root_information_gain = find_root_node(data, discrete_attributes, target_attribute)
    tree = {
        "node_attribute": root_attribute,
        "observations": dict(data[target_attribute].value_counts()),
        "information_gain": calculate_information_gain(data, root_attribute, target_attribute),
        "values": {}
    }

    # For each value of the root attribute, create a new subtree
    for value in data[root_attribute].unique():
        # Create a new subtree for the current value
        subtree = data[data[root_attribute] == value]

        # Remove the root attribute from the list of attributes
        new_attributes = [attr for attr in discrete_attributes if attr != root_attribute]

        # Recursively call the ID3 algorithm
        subtree_result = id3_discrete(subtree, new_attributes, target_attribute)

        # Add the new subtree to the existing tree
        tree["values"][value] = subtree_result

    return tree


pprint(id3_discrete(data, discrete_attributes, target_attribute))

{'information_gain': 0.010141885489185931,
 'node_attribute': 'Items_For_Spike',
 'observations': {'no': 253, 'yes': 47},
 'values': {'Defensive': {'information_gain': 0.06363963716250465,
                          'node_attribute': 'Difficulty',
                          'observations': {'no': 45, 'yes': 7},
                          'values': {'Easy': {'information_gain': 0.3632029500446188,
                                              'node_attribute': 'league',
                                              'observations': {'no': 9,
                                                               'yes': 4},
                                              'values': {'LCK': {'information_gain': 1.0,
                                                                 'node_attribute': 'result',
                                                                 'observations': {'no': 1,
                                                                                  'yes': 1},
                

## c. Run id3_discrete on the dataset containing only discrete attributes. Compare the results with the ones from sklearn . (make your comparison as 
thorough as possible)

In [13]:
X = data[discrete_attributes]
y = data[target_attribute]

# One-hot encode the discrete attributes
X = pd.get_dummies(X)

# Use sklearn's DecisionTreeClassifier
dt_classifier = DecisionTreeClassifier(criterion='entropy')
dt_classifier.fit(X, y)

tree_rules = export_text(dt_classifier, feature_names=list(X.columns))
print(tree_rules)

|--- Items_For_Spike_Support <= 0.50
|   |--- league_LEC <= 0.50
|   |   |--- result <= 0.50
|   |   |   |--- league_LCK <= 0.50
|   |   |   |   |--- jng_no <= 0.50
|   |   |   |   |   |--- class: no
|   |   |   |   |--- jng_no >  0.50
|   |   |   |   |   |--- sup_yes <= 0.50
|   |   |   |   |   |   |--- Difficulty_Easy <= 0.50
|   |   |   |   |   |   |   |--- Attack_Type_Ranged <= 0.50
|   |   |   |   |   |   |   |   |--- league_LPL <= 0.50
|   |   |   |   |   |   |   |   |   |--- Items_For_Spike_Offensive <= 0.50
|   |   |   |   |   |   |   |   |   |   |--- Difficulty_Hard <= 0.50
|   |   |   |   |   |   |   |   |   |   |   |--- class: no
|   |   |   |   |   |   |   |   |   |   |--- Difficulty_Hard >  0.50
|   |   |   |   |   |   |   |   |   |   |   |--- truncated branch of depth 2
|   |   |   |   |   |   |   |   |   |--- Items_For_Spike_Offensive >  0.50
|   |   |   |   |   |   |   |   |   |   |--- Difficulty_Hard <= 0.50
|   |   |   |   |   |   |   |   |   |   |   |--- truncated br

By comparing the output of the b. and the output of the tree_rules of the sklearn tree, we can observe that the tree has the same root node as the sklearn tree, and the same for the nodes.

## d. ## Write a function get_splits which, given a continuous attribute and the labels, will identify the splits that could be used to discretization of the variable. Test your function on an example.
For example, we will run the function on 'kills' attribute to see all the splits.

In [14]:
def get_splits(labels, attribute):
    labels = sorted(data[attribute].unique())
    splits = []
    for i in range(len(labels) - 1):
        splits.append((labels[i] + labels[i + 1]) / 2)
    return splits


labels = []
# example from my dataset
print(get_splits(labels, 'kills'))

[0.5, 1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 7.5, 8.5, 9.5, 10.5, 11.5]


## e. Write a function id3 that implements ID3 on the entire dataset, both continuous and discrete attributes. The function should return a dictionarysimilar with the one above. Compare the results with the ones from sklearn .
First, we need to calculate the information gain for continous attributes using 'get_splits' function from d.

In [15]:
def continous_information_gain(data, attribute, target_attribute, split_point):
    subset1 = data[data[attribute] <= split_point]
    subset2 = data[data[attribute] > split_point]

    p1 = len(subset1) / len(data)
    p2 = len(subset2) / len(data)

    entropy1 = calculate_entropy(subset1, target_attribute)
    entropy2 = calculate_entropy(subset2, target_attribute)

    conditional_entropy = p1 * entropy1 + p2 * entropy2
    return conditional_entropy


Then, in order to get the best outcome, we need to create another function 'find_best_split'


In [16]:
def find_best_split(data, attribute, target_attribute):
    splits = get_splits(data[attribute], attribute)
    information_gains = [(split_point, continous_information_gain(data, attribute, target_attribute, split_point)) for
                         split_point in splits]
    best_split, best_information_gain = max(information_gains, key=lambda x: x[1])
    return best_split

### Finally, we assemble the algorithm.

In [17]:
def id3(data, attributes, target_attribute):
    # if all target attributes have the same value, return that value
    if len(data[target_attribute].unique()) == 1:
        return data[target_attribute].unique()[0]

    # if data is empty, return the most common value of the target attribute
    if len(data) == 0:
        return data[target_attribute].value_counts().idxmax() if not data.empty else None

    # if there are no attributes left, return the most common target attribute value
    if len(attributes) <= 1:
        return data[target_attribute].value_counts().idxmax() if not data.empty else None

    # Choose the attribute with the highest information gain
    root_attribute, root_information_gain = find_root_node(data, attributes, target_attribute)
    if root_attribute in continuous_attributes:
        best_split = find_best_split(data, root_attribute, target_attribute)
        best_information_gain = continous_information_gain(data, root_attribute, target_attribute, best_split)
        tree = {
            "node_attribute": root_attribute,
            "observations": dict(data[target_attribute].value_counts()),
            "information_gain": best_information_gain,
            "split_point": best_split,
            "values": {}
        }

        # Create two subtrees based on the split
        for branch in [True, False]:
            subset = data[data[root_attribute] <= best_split] if branch else data[data[root_attribute] > best_split]
            subtree_result = id3(subset, attributes, target_attribute)
            tree["values"][f"branch_{branch}"] = subtree_result

    # Daca e discret
    else:
        tree = {
            "node_attribute": root_attribute,
            "observations": dict(data[target_attribute].value_counts()),
            "information_gain": root_information_gain,
            "values": {}
        }
        # For each value of the root attribute, create a new subtree
        for value in data[root_attribute].unique():
            subtree = data[data[root_attribute] == value]
            new_attributes = [attr for attr in discrete_attributes if attr != root_attribute]
            subtree_result = id3_discrete(subtree, new_attributes, target_attribute)
            tree["values"][value] = subtree_result

    return tree

Note that in the current version of this notebook, the id3 algorithm above is currently going in the maximum recursion depth.

# We create the tree from the sklearn library.

In [18]:
X = data[discrete_attributes + continuous_attributes]
y = data[target_attribute]

# One-hot encode the discrete attributes
X = pd.get_dummies(X)

# Use sklearn's DecisionTreeClassifier
dt_classifier = DecisionTreeClassifier(criterion='entropy')
dt_classifier.fit(X, y)

tree_rules = export_text(dt_classifier, feature_names=list(X.columns))
print(tree_rules)

|--- kills <= 0.50
|   |--- class: no
|--- kills >  0.50
|   |--- deaths <= 4.50
|   |   |--- assists <= 12.50
|   |   |   |--- deaths <= 2.50
|   |   |   |   |--- kills <= 7.50
|   |   |   |   |   |--- kills <= 1.50
|   |   |   |   |   |   |--- assists <= 10.50
|   |   |   |   |   |   |   |--- class: no
|   |   |   |   |   |   |--- assists >  10.50
|   |   |   |   |   |   |   |--- side_Red <= 0.50
|   |   |   |   |   |   |   |   |--- class: no
|   |   |   |   |   |   |   |--- side_Red >  0.50
|   |   |   |   |   |   |   |   |--- class: yes
|   |   |   |   |   |--- kills >  1.50
|   |   |   |   |   |   |--- damagetochampions <= 30714.00
|   |   |   |   |   |   |   |--- damagetochampions <= 30274.50
|   |   |   |   |   |   |   |   |--- damagetochampions <= 21344.00
|   |   |   |   |   |   |   |   |   |--- damagetochampions <= 20004.50
|   |   |   |   |   |   |   |   |   |   |--- Difficulty_Easy <= 0.50
|   |   |   |   |   |   |   |   |   |   |   |--- truncated branch of depth 7
|   |   

## f. Modify the two ID3 functions such that they will allow pruning. Use TWO methods of pruning, one of which should be based on the depth of the tree.
- Since the id3 from e. is not working, we will use the id3_discrete function from b.

#### Depth pruning

The function, `id3_discrete_depth_pruning`, includes depth-based pruning to limit the growth of the tree. Here's a brief summary:

##### Input:

- `data`: The dataset with discrete attributes.
- `discrete_attributes`: List of discrete attribute names.
- `target_attribute`: The target attribute to be predicted.
- `max_depth`: Maximum depth allowed for the tree (pruning parameter).
- `current_depth`: Current depth in the recursive process (used for depth-based pruning).

##### Base Cases:

- If all instances in the subset have the same target value, return that value.
- If the dataset is empty, return the most common target attribute value.
- If there are no attributes left or the maximum depth is reached, return the most common target attribute value.

##### Tree Construction:

- Choose the root attribute with the highest information gain.
- Create a node in the tree with information about the root attribute.
- For each unique value of the root attribute, create a subtree by recursively calling the function.
- Prune the tree if the maximum depth is reached.

##### Output:

- The resulting tree structure in a dictionary format.

##### Example Usage:

- Build a decision tree with discrete attributes and depth-based pruning up to a specified maximum depth (e.g., 2).
- Print or visualize the resulting tree structure.
esulting tree structure.

In [19]:
def id3_discrete_depth_pruning(data, discrete_attributes, target_attribute, max_depth=None, current_depth=0):
    if len(data[target_attribute].unique()) == 1:
        return data[target_attribute].unique()[0]
    if len(data) == 0:
        return data[target_attribute].value_counts().idxmax() if not data.empty else None

    # if there are no attributes left or reached max depth, return the most common target attribute value
    if len(discrete_attributes) <= 1 or current_depth == max_depth:
        return data[target_attribute].value_counts().idxmax() if not data.empty else None

    root_attribute, root_information_gain = find_root_node(data, discrete_attributes, target_attribute)
    tree = {
        "node_attribute": root_attribute,
        "observations": dict(data[target_attribute].value_counts()),
        "information_gain": calculate_information_gain(data, root_attribute, target_attribute),
        "values": {}
    }

    for value in data[root_attribute].unique():
        subtree = data[data[root_attribute] == value]
        new_attributes = [attr for attr in discrete_attributes if attr != root_attribute]
        subtree_result = id3_discrete_depth_pruning(subtree, new_attributes, target_attribute, max_depth=max_depth,
                                                    current_depth=current_depth + 1)
        tree["values"][value] = subtree_result

    return tree


result_tree = id3_discrete_depth_pruning(data, discrete_attributes, target_attribute, max_depth=2)
pprint(result_tree)

{'information_gain': 0.010141885489185931,
 'node_attribute': 'Items_For_Spike',
 'observations': {'no': 253, 'yes': 47},
 'values': {'Defensive': {'information_gain': 0.06363963716250465,
                          'node_attribute': 'Difficulty',
                          'observations': {'no': 45, 'yes': 7},
                          'values': {'Easy': 'no',
                                     'Hard': 'no',
                                     'Medium': 'no'}},
            'Offensive': {'information_gain': 0.007475328969699491,
                          'node_attribute': 'league',
                          'observations': {'no': 160, 'yes': 36},
                          'values': {'LCK': 'no',
                                     'LCS': 'no',
                                     'LEC': 'no',
                                     'LPL': 'no'}},
            'Support': {'information_gain': 0.06441115600257952,
                        'node_attribute': 'result',
                        '

#### Observation pruning

The function, `id3_discrete_observation_pruning`, includes observation-based pruning to limit the growth of the tree. Here's a brief summary:

##### Input:

- `data`: The dataset with discrete attributes.
- `discrete_attributes`: List of discrete attribute names.
- `target_attribute`: The target attribute to be predicted.
- `min_obs`: Minimum number of observations required for a subset to be considered for further splitting.

##### Base Cases:

- If all instances in the subset have the same target value, return that value.
- If the dataset is empty or the number of observations is less than `min_obs`, return the most common target attribute value.

##### Tree Construction:

- Choose the root attribute with the highest information gain.
- Create a node in the tree with information about the root attribute.
- For each unique value of the root attribute, create a subtree by recursively calling the function.
- Prune the tree if the number of observations is less than `min_obs`.

##### Output:

- The resulting tree structure in a dictionary format.

##### Example Usage:

- Build a decision tree with discrete attributes and observation-based pruning, specifying the minimum2observations (e.g., min_obs=3).
- Print or visualize the resulting tree structure.


In [20]:
def id3_discrete_observation_pruning(data, discrete_attributes, target_attribute, min_obs):
    # if all target attributes have the same value, return that value
    if len(data[target_attribute].unique()) == 1:
        return data[target_attribute].unique()[0]

    # if data is empty or observations are less than min_obs, return the most common value of the target attribute
    if len(data) == 0 or len(data) < min_obs:
        return data[target_attribute].value_counts().idxmax() if not data.empty else None

    # if there are no attributes left, return the most common target attribute value
    if len(discrete_attributes) <= 1:
        return data[target_attribute].value_counts().idxmax() if not data.empty else None

    # Choose the attribute with the highest information gain
    root_attribute, root_information_gain = find_root_node(data, discrete_attributes, target_attribute)
    tree = {
        "node_attribute": root_attribute,
        "observations": dict(data[target_attribute].value_counts()),
        "information_gain": calculate_information_gain(data, root_attribute, target_attribute),
        "values": {}
    }

    # For each value of the root attribute, create a new subtree
    for value in data[root_attribute].unique():
        # Create a new subtree for the current value
        subtree = data[data[root_attribute] == value]

        # Remove the root attribute from the list of attributes
        new_attributes = [attr for attr in discrete_attributes if attr != root_attribute]

        # Recursively call the ID3 algorithm
        subtree_result = id3_discrete_observation_pruning(subtree, new_attributes, target_attribute, min_obs)

        # Add the new subtree to the existing tree
        tree["values"][value] = subtree_result

    return tree


# Call the modified id3 function
result_tree = id3_discrete_observation_pruning(data, discrete_attributes, target_attribute, min_obs=2)
pprint(result_tree)

{'information_gain': 0.010141885489185931,
 'node_attribute': 'Items_For_Spike',
 'observations': {'no': 253, 'yes': 47},
 'values': {'Defensive': {'information_gain': 0.06363963716250465,
                          'node_attribute': 'Difficulty',
                          'observations': {'no': 45, 'yes': 7},
                          'values': {'Easy': {'information_gain': 0.3632029500446188,
                                              'node_attribute': 'league',
                                              'observations': {'no': 9,
                                                               'yes': 4},
                                              'values': {'LCK': {'information_gain': 1.0,
                                                                 'node_attribute': 'result',
                                                                 'observations': {'no': 1,
                                                                                  'yes': 1},
                